## Load Dataset

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
# libgomp issue, must import n2 before torch
from n2 import HnswIndex

In [4]:
import sys

sys.path.insert(0, '../..')

In [5]:
import os
home_dir = os.getenv('HOME')

https://dbs.uni-leipzig.de/research/projects/object_matching/benchmark_datasets_for_entity_resolution

In [6]:
import glob
import csv
from tqdm.auto import tqdm

from entity_embed.data_utils.utils import Enumerator

row_dict = {}
id_enumerator = Enumerator()
left_id_set = set()
right_id_set = set()
rows_total = 1363 + 3226
clusters_total = 1300

with tqdm(total=rows_total) as pbar:
    with open(f'{home_dir}/Downloads/amazon_google_exp_data/tableA.csv') as f:
        for row in csv.DictReader(f):
            row['id'] = id_enumerator[f'left-{int(row["id"])}']
            row['source'] = 'left'
            row_dict[row['id']] = row
            left_id_set.add(row['id'])
            pbar.update(1)
    
    with open(f'{home_dir}/Downloads/amazon_google_exp_data/tableB.csv') as f:
        for row in csv.DictReader(f):
            row['id'] = id_enumerator[f'right-{int(row["id"])}']
            row['source'] = 'right'
            row_dict[row['id']] = row
            right_id_set.add(row['id'])
            pbar.update(1)

In [7]:
train_true_pair_set = set()
valid_true_pair_set = set()
test_true_pair_set = set()

with open(f'{home_dir}/Downloads/amazon_google_exp_data/train.csv') as f:
    for row in csv.DictReader(f):
        if int(row['label']) == 1:
            id_left = id_enumerator[f'left-{int(row["ltable_id"])}']
            id_right = id_enumerator[f'right-{int(row["rtable_id"])}']
            train_true_pair_set.add((id_left, id_right))

with open(f'{home_dir}/Downloads/amazon_google_exp_data/valid.csv') as f:
    for row in csv.DictReader(f):
        if int(row['label']) == 1:
            id_left = id_enumerator[f'left-{int(row["ltable_id"])}']
            id_right = id_enumerator[f'right-{int(row["rtable_id"])}']
            valid_true_pair_set.add((id_left, id_right))

with open(f'{home_dir}/Downloads/amazon_google_exp_data/test.csv') as f:
    for row in csv.DictReader(f):
        if int(row['label']) == 1:
            id_left = id_enumerator[f'left-{int(row["ltable_id"])}']
            id_right = id_enumerator[f'right-{int(row["rtable_id"])}']
            test_true_pair_set.add((id_left, id_right))
        
display(('train_true_pair_set', len(train_true_pair_set)))
display(('valid_true_pair_set', len(valid_true_pair_set)))
display(('test_true_pair_set', len(test_true_pair_set)))

('train_true_pair_set', 699)

('valid_true_pair_set', 234)

('test_true_pair_set', 234)

## Preprocess

In [8]:
attr_list = ['title', 'manufacturer', 'price']

In [9]:
import unidecode
import itertools
from entity_embed import default_tokenizer

def clean_str(s):
    s = unidecode.unidecode(s).lower().strip()
    s_tokens = itertools.islice((s_part[:30] for s_part in default_tokenizer(s)), 0, 30)
    return ' '.join(s_tokens)[:300]

for row in tqdm(row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

## Init Data Module

In [10]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [11]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [12]:
attr_info_dict = {
    'title': {
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
    },
    'manufacturer': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'price': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    }
}

In [13]:
from entity_embed import build_row_numericalizer

row_numericalizer = build_row_numericalizer(attr_info_dict, row_dict=row_dict)
row_numericalizer.attr_info_dict

15:46:02 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
15:46:11 INFO:For attr='manufacturer', computing actual alphabet and max_str_len
15:46:11 INFO:actual_max_str_len=15 must be pair to enable NN pooling. Updating to 16
15:46:11 INFO:For attr='manufacturer', using actual_max_str_len=16
15:46:11 INFO:For attr='price', computing actual alphabet and max_str_len
15:46:11 INFO:actual_max_str_len=11 must be pair to enable NN pooling. Updating to 12
15:46:11 INFO:For attr='price', using actual_max_str_len=12


{'title': NumericalizeInfo(field_type=<FieldType.SEMANTIC_MULTITOKEN: 'semantic_multitoken'>, tokenizer=<function default_tokenizer at 0x7f29486bfca0>, alphabet=None, max_str_len=None, vocab=<torchtext.vocab.Vocab object at 0x7f29c5ff9640>),
 'manufacturer': NumericalizeInfo(field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer=<function default_tokenizer at 0x7f29486bfca0>, alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=16, vocab=None),
 'price': NumericalizeInfo(field_type=<FieldType.STRING: 'string'>, tokenizer=None, alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r

In [14]:
from entity_embed import PairwiseDataModule

datamodule = PairwiseDataModule(
    row_dict=row_dict,
    row_numericalizer=row_numericalizer,
    batch_size=20,
    row_batch_size=16,
    train_true_pair_set=train_true_pair_set,
    valid_true_pair_set=valid_true_pair_set,
    test_true_pair_set=test_true_pair_set,
    random_seed=random_seed
)

## Training

In [15]:
from entity_embed import LinkageEmbed

ann_k = 100
model = LinkageEmbed(
    datamodule,
    ann_k=ann_k,
    use_mask=True,
    embedding_size=300
)

In [16]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 100
early_stop_callback = EarlyStopping(
   monitor='valid_f1_at_0.7',
   min_delta=0.00,
   patience=20,
   verbose=True,
   mode='max'
)
tb_log_dir = '../tb_logs'
tb_name = 'f1-amzn-googl'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_log_dir, name=tb_name)
)

15:46:12 INFO:GPU available: True, used: True
15:46:12 INFO:TPU available: None, using: 0 TPU cores
15:46:12 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [17]:
trainer.fit(model, datamodule)

15:46:17 INFO:
  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 4.8 M 
1 | losser      | SupConLoss | 0     
-------------------------------------------
3.1 M     Trainable params
1.7 M     Non-trainable params
4.8 M     Total params


15:46:19 WARNING:true_pair_set has 82 less elements than transitive_true_pair_set. This means there are transitive true pairs not included in true_pair_set.


1

In [18]:
model.blocker_net.get_signature_weights()

{'title': 0.5434083938598633,
 'manufacturer': 0.17624834179878235,
 'price': 0.2803432047367096}

In [19]:
from entity_embed import validate_best

validate_best(trainer)

{'valid_f1_at_0.3': 0.3961605584642233,
 'valid_f1_at_0.5': 0.7184466019417476,
 'valid_f1_at_0.7': 0.8,
 'valid_f1_at_0.9': 0.6984924623115578,
 'valid_pair_entity_ratio_at_0.3': 1.9956236323851204,
 'valid_pair_entity_ratio_at_0.5': 0.8402625820568927,
 'valid_pair_entity_ratio_at_0.7': 0.6367614879649891,
 'valid_pair_entity_ratio_at_0.9': 0.3588621444201313,
 'valid_precision_at_0.3': 0.24890350877192982,
 'valid_precision_at_0.5': 0.578125,
 'valid_precision_at_0.7': 0.7216494845360825,
 'valid_precision_at_0.9': 0.8475609756097561,
 'valid_recall_at_0.3': 0.9700854700854701,
 'valid_recall_at_0.5': 0.9487179487179487,
 'valid_recall_at_0.7': 0.8974358974358975,
 'valid_recall_at_0.9': 0.594017094017094}

## Testing

In [20]:
trainer.test(ckpt_path='best', verbose=False)

[{'test_f1_at_0.3': 0.35901509134233517,
  'test_f1_at_0.5': 0.7058823529411764,
  'test_f1_at_0.7': 0.8007662835249041,
  'test_f1_at_0.9': 0.6666666666666667,
  'test_pair_entity_ratio_at_0.3': 2.2234273318872018,
  'test_pair_entity_ratio_at_0.5': 0.8568329718004338,
  'test_pair_entity_ratio_at_0.7': 0.6247288503253796,
  'test_pair_entity_ratio_at_0.9': 0.2993492407809111,
  'test_precision_at_0.3': 0.22048780487804878,
  'test_precision_at_0.5': 0.5620253164556962,
  'test_precision_at_0.7': 0.7256944444444444,
  'test_precision_at_0.9': 0.8985507246376812,
  'test_recall_at_0.3': 0.9658119658119658,
  'test_recall_at_0.5': 0.9487179487179487,
  'test_recall_at_0.7': 0.8931623931623932,
  'test_recall_at_0.9': 0.5299145299145299}]

## Testing manually 

In [21]:
from entity_embed.data_utils.utils import id_pairs_to_cluster_mapping_and_dict, cluster_dict_to_filtered_row_dict

__, test_cluster_dict = id_pairs_to_cluster_mapping_and_dict(test_true_pair_set)
test_row_dict = cluster_dict_to_filtered_row_dict(row_dict, test_cluster_dict)
test_left_vector_dict, test_right_vector_dict = model.predict(
    row_dict=test_row_dict,
    left_id_set=left_id_set,
    right_id_set=right_id_set,
    batch_size=16
)

In [22]:
embedding_size = model.blocker_net.embedding_size

In [23]:
assert (len(test_left_vector_dict) + len(test_right_vector_dict)) == len(test_row_dict)

In [24]:
%%time

from entity_embed import ANNLinkageIndex

ann_index = ANNLinkageIndex(embedding_size=embedding_size)
ann_index.insert_vector_dict(left_vector_dict=test_left_vector_dict, right_vector_dict=test_right_vector_dict)
ann_index.build()

CPU times: user 540 ms, sys: 0 ns, total: 540 ms
Wall time: 64.2 ms


In [25]:
%%time

sim_threshold = 0.7
found_pair_set = ann_index.search_pairs(
    k=ann_k,
    sim_threshold=sim_threshold,
    left_vector_dict=test_left_vector_dict,
    right_vector_dict=test_right_vector_dict,
)

CPU times: user 405 ms, sys: 0 ns, total: 405 ms
Wall time: 42.3 ms


In [26]:
from entity_embed.evaluation import pair_entity_ratio

pair_entity_ratio(len(found_pair_set), len(test_row_dict))

0.6247288503253796

In [27]:
from entity_embed.evaluation import precision_and_recall, f1_score

precision, recall = precision_and_recall(found_pair_set, test_true_pair_set)
precision, recall

(0.7256944444444444, 0.8931623931623932)

In [28]:
f1_score(precision, recall)

0.8007662835249041

In [29]:
false_positives = list(found_pair_set - test_true_pair_set)
len(false_positives)

79

In [30]:
false_negatives = list(test_true_pair_set - found_pair_set)
len(false_negatives)

25

In [31]:
cos_similarity = lambda a, b: np.dot(a, b)

In [32]:
for (id_left, id_right) in false_negatives[:10]:
    display(
        (
            cos_similarity(test_left_vector_dict[id_left], test_right_vector_dict[id_right]),
            row_dict[id_left], row_dict[id_right]
        )
    )

(0.6632247,
 {'id': 17,
  'title': 'upg sgms 1000 incremental node',
  'manufacturer': 'sonic - systems - inc .',
  'price': '',
  'source': 'left'},
 {'id': 3347,
  'title': 'sonicwall gms 1000n - incremental lic upg',
  'manufacturer': 'sonic - systems - inc .',
  'price': '62920 . 89',
  'source': 'right'})

(0.5919264,
 {'id': 395,
  'title': 'world history ( win / mac ) ( jewel case )',
  'manufacturer': 'fogware publishing',
  'price': '9 . 99',
  'source': 'left'},
 {'id': 4269,
  'title': 'high school world history ( pc / mac ) fogware',
  'manufacturer': '',
  'price': '9 . 99',
  'source': 'right'})

(0.4314089,
 {'id': 207,
  'title': 'crystal reports 11 professional full product french',
  'manufacturer': 'business objects',
  'price': '909 . 66',
  'source': 'left'},
 {'id': 2888,
  'title': 'crystal reports xi professional edition complete package',
  'manufacturer': '',
  'price': '537 . 57',
  'source': 'right'})

(0.24009371,
 {'id': 1206,
  'title': 'contentbarrier x4 10 . 4 single user ( mac )',
  'manufacturer': 'intego',
  'price': '49 . 99',
  'source': 'left'},
 {'id': 1900,
  'title': 'intego contentbarrier x4 10 . 4',
  'manufacturer': '',
  'price': '54 . 99',
  'source': 'right'})

(0.41482317,
 {'id': 1360,
  'title': 'dragon naturally speaking standard v9',
  'manufacturer': 'nuance - communications - inc .',
  'price': '99 . 99',
  'source': 'left'},
 {'id': 1973,
  'title': 'nuance communications inc . dragon ns standard v9',
  'manufacturer': 'nuance - communications - inc .',
  'price': '92 . 51',
  'source': 'right'})

(0.5419756,
 {'id': 1190,
  'title': 'punch ! professional home design',
  'manufacturer': 'punch ! software',
  'price': '89 . 99',
  'source': 'left'},
 {'id': 3789,
  'title': 'punch software 14100 punch ! home design suite professional architectural 1 user ( s ) pc',
  'manufacturer': '',
  'price': '59 . 97',
  'source': 'right'})

(0.6443166,
 {'id': 161,
  'title': 'gods : lands of infinity special edition',
  'manufacturer': 'strategy first',
  'price': '29 . 99',
  'source': 'left'},
 {'id': 2854,
  'title': 'gods : lands of infinity spec',
  'manufacturer': '',
  'price': '16 . 99',
  'source': 'right'})

(0.45422032,
 {'id': 273,
  'title': 'quickbooks credit card processing kit 3 . 0',
  'manufacturer': 'intuit',
  'price': '39 . 95',
  'source': 'left'},
 {'id': 1371,
  'title': 'qb credit card kit 3 . 0',
  'manufacturer': 'intuit',
  'price': '25 . 99',
  'source': 'right'})

(0.23871195,
 {'id': 726,
  'title': 'simply put software data eliminator',
  'manufacturer': 'simply - put - software',
  'price': '',
  'source': 'left'},
 {'id': 4151,
  'title': 'simply put software llc de905 - s data elminator ( win 95 98 me nt 2000 xp )',
  'manufacturer': 'simply - put - software',
  'price': '34 . 97',
  'source': 'right'})

(0.18454044,
 {'id': 812,
  'title': 'extensis smartscale 1 - user pxe - 11433 )',
  'manufacturer': 'extensis corporation',
  'price': '',
  'source': 'left'},
 {'id': 4160,
  'title': 'onone software pxe - 11433 pxl smartscale elect 1 - user english mac / win',
  'manufacturer': 'onone software',
  'price': '158 . 99',
  'source': 'right'})